In [1]:
import wandb,os
from fastcore.all import *
import pandas as pd
from fastai.vision.all import *
import plotly.express as px

In [2]:
os.environ['WANDB_API_KEY'] = 'ac83272eb92a7df88c296f179996d615edda0dd3'

In [5]:
api = wandb.Api()

sweep_ids = [
    'paddy-ft/sweeps/0mxkdioz',
]

sweeps = concat(api.sweep(o).runs for o in sweep_ids)
summs = [{**r.summary, 'model_name':r.config['model_name']} for r in sweeps]


AttributeError: 'Runs' object has no attribute 'summary'

In [6]:

df = pd.DataFrame(summs)
df['error_rate'] = 1-df.accuracy


NameError: name 'summs' is not defined

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
cols = ['model_name', 'GPU_mem', 'error_rate', 'valid_loss', 'train_loss', 'fit_time']
df[cols].to_csv('sweeps.csv', index=False)

In [ ]:
df = pd.read_csv('sweeps.csv')

In [ ]:
df['family'] = df.model_name.str.extract('^([a-z]+?(?:v2)?)(?:\d|_|$)')
df.loc[df.family=='swinv2', 'family'] = 'swin'
pt_all = df.pivot_table(values=['error_rate','fit_time','GPU_mem'], index=['family', 'model_name'],
                    aggfunc=np.min).reset_index()
pt_all['score'] = pt_all.error_rate*(pt_all.fit_time+80)

In [ ]:
pt = pt_all.sort_values('score').reset_index(drop=True)
pt.head(15)

In [ ]:
w,h = 900,700
faves = ['efficientnet','convnext']
pt2 = pt[pt.family.isin(faves)]
px.scatter(pt2, width=w, height=h, x='fit_time', y='error_rate', color='family', hover_name='model_name', trendline="ols",)